# Predicting Cardiovascular Disease

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from path import Path
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import sqlalchemy
from config import db_password

C:\Users\zzyzz\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zzyzz\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zzyzz\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zzyzz\anaconda3\envs\PythonData\lib\site-packages

In [3]:
# Import csv file 
data = Path('/Users/Maryam/Desktop/utor-virt-data-pt-06-2020-u-b/cardio_final_project/cardio_train.csv')
df = pd.read_csv(data)
df.head()

FileNotFoundError: [Errno 2] File /Users/Maryam/Desktop/utor-virt-data-pt-06-2020-u-b/cardio_final_project/cardio_train.csv does not exist: Path('/Users/Maryam/Desktop/utor-virt-data-pt-06-2020-u-b/cardio_final_project/cardio_train.csv')

 ## Clean the data


In [ ]:
df.set_index('id', inplace=True)
df

In [ ]:
# Columns
df.columns

In [ ]:
#Type of data
df.dtypes

In [ ]:
#Find null values
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values" )
    

In [ ]:
# Determine the number of unique values in each column.
df.nunique()

## Declare target and feature variables and split data into training and testing

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df.cardio
X = df.drop(columns=["cardio"])


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

## Scale the data

In [4]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

NameError: name 'X_train' is not defined

## Compile, Train and Evaluate the Model

In [56]:
# Define the model - deep neural network, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  30
hidden_nodes_layer2 = 24
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                360       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                744       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 25        
Total params: 1,129
Trainable params: 1,129
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

## Train Model

In [58]:
# Train the model
y_train=y_train.to_numpy()
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Train on 52500 samples
Epoch 1/100
52500/52500 [==============================] - 6s 123us/sample - loss: 0.6076 - accuracy: 0.6741
Epoch 2/100
52500/52500 [==============================] - 5s 92us/sample - loss: 0.5650 - accuracy: 0.7261
Epoch 3/100
52500/52500 [==============================] - 6s 122us/sample - loss: 0.5549 - accuracy: 0.7309
Epoch 4/100
52500/52500 [==============================] - 5s 94us/sample - loss: 0.5502 - accuracy: 0.7332
Epoch 5/100
52500/52500 [==============================] - 6s 113us/sample - loss: 0.5466 - accuracy: 0.7333
Epoch 6/100
52500/52500 [==============================] - 6s 115us/sample - loss: 0.5453 - accuracy: 0.7342
Epoch 7/100
52500/52500 [==============================] - 6s 110us/sample - loss: 0.5443 - accuracy: 0.7335
Epoch 8/100
52500/52500 [==============================] - 5s 96us/sample - loss: 0.5425 - accuracy: 0.7347
Epoch 9/100
52500/52500 [==============================] - 6s 107us/sample - loss: 0.5420 - accuracy: 0.7358

In [59]:
# Evaluate the model using the test data
y_test= y_test.to_numpy()
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

17500/1 - 1s - loss: 0.5829 - accuracy: 0.7317
Loss: 0.5478981971876962, Accuracy: 0.7316571474075317


In [1]:
predictions = model.predict(X_test_scaled)

NameError: name 'model' is not defined

In [ ]:
# Export our model to HDF5 file
nn.save("CardiovascularPredictions.h5")